In [1]:

import os
import os.path as osp
import json
import collections as C
import itertools as I
import random
import pickle
from typing import List
import regex as re

import aiofiles
import pyarrow
import pyarrow.parquet as pq

from common.constants import MVAR_PATTERN
from common.utils import remove_comments, parse_idents, zip_strict, normalize_spaces, remove_spaces
from common.constants import OPEN_HEADER, CORE_OPTIONS
from common.pantograph.dataclasses import TacticInvocation
from common.pantograph.server import PersistentServer, Server
from common.pantograph.solving_server import PersistentBaseSolvingServer

In [2]:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Numina-Lean/20250815-172942.pkl', 'rb') as f:
    invocations_all = pickle.load(f)

In [3]:
def match_wo_mvar(s_w_mvar: str, s_wo_mvar: str) -> str:
    s_w_mvar = normalize_spaces(s_w_mvar.replace('(', '').replace(')', ''))
    s_wo_mvar = normalize_spaces(s_wo_mvar.replace('(', '').replace(')', ''))
    parts = re.split(MVAR_PATTERN, s_w_mvar)
    pattern = '^' + '.*?'.join(re.escape(part) for part in parts) + '.*?$'
    match = re.match(pattern, s_wo_mvar)
    return bool(match)

In [4]:
data = pq.read_table('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/train-00000-of-00001.parquet').to_pandas().to_dict(orient="records")

In [5]:
len(data)

104155

In [6]:
for d in data:
    if isinstance(d['formal_ground_truth'], str) and len(d['formal_ground_truth']) == 0 or d['ground_truth_type'] != 'complete':
        d['formal_ground_truth'] = None
    if isinstance(d['formal_proof'], str) and len(d['formal_proof']) == 0:
        d['formal_proof'] = None

In [7]:
data[0].keys()

dict_keys(['uuid', 'problem', 'question_type', 'answer', 'author', 'formal_statement', 'formal_ground_truth', 'ground_truth_type', 'formal_proof', 'rl_data', 'source', 'problem_type', 'exam'])

In [8]:
data[0]['problem_type']

'Algebra'

In [9]:
C.Counter(d['question_type'] for d in data)

Counter({'math-word-problem': 56786,
         'proof': 32069,
         'MCQ': 8401,
         '': 6896,
         None: 3})

In [10]:
C.Counter(d['source'] for d in data)

Counter({'olympiads': 51373,
         'unknown': 14266,
         'aops_forum': 12955,
         'secondary_math': 9551,
         'inequalities': 4309,
         'olympiads_ref': 3311,
         'math_train': 2156,
         'amc_aime': 1875,
         'math_test': 1490,
         'number_theory': 1420,
         'synthetic': 692,
         'cn_k12': 400,
         'cn_contest': 357})

In [11]:
C.Counter(d['problem_type'] for d in data)

Counter({'Algebra': 39678,
         'Number Theory': 28559,
         'Inequalities': 14776,
         'unknown': 14172,
         'Calculus': 4506,
         'Other': 1053,
         'NaN': 929,
         'Combinatorics': 349,
         'Geometry': 67,
         'Logic and Puzzles': 27,
         'Intermediate Algebra': 8,
         'Arithmetic': 6,
         'Functional Equations': 6,
         'Linear Algebra': 6,
         'Precalculus': 3,
         'Trigonometry': 3,
         'Recursion Other': 2,
         'Prealgebra': 2,
         'Recurrence Relations': 1,
         'Statistics': 1,
         'Graph Theory': 1})

In [12]:
data[0].keys()

dict_keys(['uuid', 'problem', 'question_type', 'answer', 'author', 'formal_statement', 'formal_ground_truth', 'ground_truth_type', 'formal_proof', 'rl_data', 'source', 'problem_type', 'exam'])

In [13]:
datapoints = []
for d in data:
    if d['formal_ground_truth'] is not None:
        datapoints.append({
            k : v for (k, v) in d.items() if k != 'formal_ground_truth' and k != 'formal_proof'
        } | {'formal_code' : d['formal_ground_truth']})
    if d['formal_proof'] is not None:
        datapoints.append({
            k : v for (k, v) in d.items() if k != 'formal_ground_truth' and k != 'formal_proof'
        } | {'formal_code' : d['formal_proof']})
len(datapoints)

41109

In [14]:
d = random.choice(datapoints)
print(d['problem'])
print(d['answer'])
print('-----')
print(d['formal_statement'])
print('-----')
print(d['formal_code'])

A geometric sequence starts $16$, $-24$, $36$, $-54$. What is the common ratio of this sequence?
unknown
-----
import Mathlib

/- A geometric sequence starts $16$, $-24$, $36$, $-54$. What is the common ratio of this sequence? -/
theorem algebra_11320 (a r : ℚ) (h₀ : a = 16) (h₁ : r ≠ 0)
    (h₂ : a * r = -24) (h₃ : -24 * r = 36) (h₄ : 36 * r = -54) :
    r = -3 / 2 := by
-----
import Mathlib

theorem algebra_11320 (a r : ℚ) (h₀ : a = 16) (h₁ : r ≠ 0)
    (h₂ : a * r = -24) (h₃ : -24 * r = 36) (h₄ : 36 * r = -54) :
    r = -3 / 2 := by
  rw [h₀] at h₂ -- rewrite a as 16 in h₂
  linarith -- solves linear equations


In [15]:
import_cnt = C.Counter()
open_cnt = C.Counter()
open_scoped_cnt = C.Counter()
option_cnt = C.Counter()

ood_lines = C.Counter()
parsed_datapoints = []
failed_datapoints = []

for d in datapoints:
    p_raw = d['formal_code']
    try:
        p = remove_comments(p_raw).strip().replace('\nlemma ', '\ntheorem ').replace('\nexample ', '\ntheorem thm_example')
        start_pos = p.find('theorem')
        assert start_pos != -1
        intro, stmt = p[:start_pos], p[start_pos:]
        for l in intro.splitlines():
            if len(l.strip()) == 0:
                continue
            elif l.startswith('import '):
                import_cnt[l[len('import '):].strip()] += 1
            elif l.startswith('open scoped '):
                for t in l[len('open scoped '):].strip().split():
                    open_scoped_cnt[t] += 1
            elif l.startswith('open '):
                for t in l[len('open '):].strip().split():
                    open_cnt[t] += 1
            elif l.startswith('set_option '):
                option_cnt[l[len('set_option '):].strip()] += 1
            else:
                raise
        parsed_datapoints.append(d)
    except:
        failed_datapoints.append(d)
        continue

print(len(parsed_datapoints), len(failed_datapoints))

39989 1120


In [16]:
# import_cnt = C.Counter()
# open_cnt = C.Counter()
# open_scoped_cnt = C.Counter()
# option_cnt = C.Counter()

# stop_token_cnt = C.Counter()

# for p_raw in stmt_proofs:
#     p = remove_comments(p_raw).strip()
#     lines = p.splitlines()
    
#     namespaces_to_remove = []
#     for l in lines:
#         if l.startswith('namespace '):
#             namespaces_to_remove.append(l[len('namespace '):])
#     lines = [
#         l for l in lines if not any(
#             l.startswith(f'namespace {t}') or l.startswith(f'open {t}') or l.startswith(f'open scoped {t}') or l.startswith(f'end {t}') for t in namespaces_to_remove
#         )
#     ]
            
    
#     if lines[-1].startswith('open '):   # Formatting
#         lines.pop(-1)
#     for i, l in enumerate(lines):
#         if len(l.strip()) == 0:
#             continue
#         elif l.startswith('import '):
#             import_cnt[l[len('import '):].strip()] += 1
#         elif l.startswith('open scoped '):
#             for t in l[len('open scoped '):].strip().split():
#                 open_scoped_cnt[t] += 1
#         elif l.startswith('open '):
#             for t in l[len('open '):].strip().split():
#                 open_scoped_cnt[t] += 1
#         elif l.startswith('set_option '):
#             option_cnt[l[len('set_option '):].strip()] += 1
#         else:
#             stop_token_cnt[l.split()[0]] += 1
#             break
#     if l.split()[0] in ['def', 'inductive', 'abbrev']:
#         continue
#     for t in ['import ', 'open ', 'set_option']:
#         for j, l in enumerate(lines[i+1:]):
#             if l.startswith(t):
#                 print(i+1+j, l)
#                 raise

In [75]:
# server = PersistentBaseSolvingServer(
#     imports=["Mathlib", "Aesop"],
#     project_path='/home/ma-user/workspace/fps_pantograph/formal_problem_solving/data/MiniF2F',
#     timeout=300,
#     tag='test',
#     _sync_init=False,
# )

server = PersistentServer(
    is_state_based=True,
    tag='test',
    _sync_init=False,
    imports=["Mathlib", "Aesop"],
    project_path='/home/ma-user/workspace/fps_pantograph/formal_problem_solving/data/MiniF2F',
    core_options=CORE_OPTIONS,
    timeout=300,
)

# server = await Server.create(
#     imports=["Mathlib", "Aesop"],
#     project_path='/home/ma-user/workspace/fps_pantograph/formal_problem_solving/data/MiniF2F',
#     core_options=CORE_OPTIONS,
#     timeout=300,
#     start=True,
# )

In [87]:
# Ensure the order of `invocations_all` and `parsed_datapoints` matches
assert len(invocations_all) == len(parsed_datapoints)

data_success = []

for idx, (ivc_all, d) in enumerate(zip(invocations_all, parsed_datapoints)):
    try:
        p_raw = d['formal_code']
        p = remove_comments(p_raw).strip().replace('\nlemma ', '\ntheorem ').replace('\nexample ', '\ntheorem thm_example')
        start_pos = p.find('theorem')
        assert start_pos != -1
        intro, stmt = p[:start_pos], p[start_pos:]

        import_list = []
        open_scoped_list = []
        open_list = []
        option_list = []

        for l in intro.splitlines():
            if len(l.strip()) == 0:
                continue
            elif l.startswith('import '):
                import_list.append(l[len('import '):].strip())
            elif l.startswith('open scoped '):
                for t in l[len('open scoped '):].strip().split():
                    open_scoped_list.append(t)
            elif l.startswith('open '):
                for t in l[len('open '):].strip().split():
                    open_list.append(t)
            elif l.startswith('set_option '):
                option_list.append(l[len('set_option '):].strip())
            else:
                raise
        
        assert open_scoped_list == ivc_all['open_scoped_list']
        assert open_list == ivc_all['open_list']
        assert option_list == ivc_all['option_list']
        
        ivc_all['import_list'] = import_list    # A bug in `numina-lean.parse.py` (0815)
        
        data_success.append(
            d | {'parse_result': ivc_all, 'index': idx}
        )
    except:
        continue

print(len(data_success))

38987


In [19]:
# idx = random.choice(range(len(parsed_datapoints)))

# d = parsed_datapoints[idx]
# p_raw = d['formal_code']
# p = remove_comments(p_raw).strip().replace('\nlemma ', '\ntheorem ').replace('\nexample ', '\ntheorem thm_example')
# start_pos = p.find('theorem')
# assert start_pos != -1
# intro, stmt = p[:start_pos], p[start_pos:]

# import_list = []
# open_scoped_list = []
# open_list = []
# option_list = []

# for l in intro.splitlines():
#     if len(l.strip()) == 0:
#         continue
#     elif l.startswith('import '):
#         import_list.append(l[len('import '):].strip())
#     elif l.startswith('open scoped '):
#         for t in l[len('open scoped '):].strip().split():
#             open_scoped_list.append(t)
#     elif l.startswith('open '):
#         for t in l[len('open '):].strip().split():
#             open_list.append(t)
#     elif l.startswith('set_option '):
#         option_list.append(l[len('set_option '):].strip())
#     else:
#         raise

In [ ]:

# d = random.choice(data_success)

In [96]:
code = r'''/--
There exists some positive integer `k` such that
`49 ∣ choose (2k, k)`.
We verify this by explicitly choosing `k = 25`.
-/
lemma exist_sol : ∃ k, 49 ∣ choose (2 * k) k := by
  use 25
  decide  -- `choose (50, 25) % 49 = 0` evaluates to true'''

for d in data_success:
    if code in d['formal_code']:
        raise

RuntimeError: No active exception to reraise

In [97]:
p_raw = d['formal_code']
p = remove_comments(p_raw).strip().replace('\nlemma ', '\ntheorem ').replace('\nexample ', '\ntheorem thm_example')
start_pos = p.find('theorem')
assert start_pos != -1
intro, stmt = p[:start_pos], p[start_pos:]

print(p)


import Mathlib

open Nat


theorem exist_sol : ∃ k, 49 ∣ choose (2 * k) k := by
  use 25
  decide  


theorem lowerbound : ∀ k : Fin 25, ¬ 49 ∣ choose (2 * k) k := by
  intro k
  fin_cases k <;> decide  


noncomputable def smallestKDivChoose : ℕ :=
  Nat.find (by
    
    exact exist_sol
  )


theorem smallest_k_choose_div_49 : smallestKDivChoose = 25 := by
  
  rw [smallestKDivChoose]
  
  apply (Nat.find_eq_iff exist_sol).mpr
  constructor
  · 
    decide
  · 
    intro n hn
    exact lowerbound ⟨n, hn⟩


In [98]:
# print('\n'.join('import ' + t for t in import_list))

import_list = d['parse_result']['import_list']
open_scoped_list = d['parse_result']['open_scoped_list']
open_list = d['parse_result']['open_list']
option_list = d['parse_result']['option_list']

print(import_list)
tactic_header = ''
load_header = ''
if len(open_scoped_list):
    tactic_header += 'open scoped ' + ' '.join(t for t in open_scoped_list) + ' in\n'
    load_header += 'open scoped ' + ' '.join(t for t in open_scoped_list) + '\n'
if len(open_list):
    tactic_header += 'open ' + ' '.join(t for t in open_list) + ' in\n'
    load_header += 'open ' + ' '.join(t for t in open_list) + '\n'
if len(option_list):
    tactic_header += '\n'.join('set_option ' + t for t in option_list + ' in') + '\n'
    load_header += '\n'.join('set_option ' + t for t in option_list) + '\n'
print(tactic_header)

['Mathlib']
open Nat in



In [99]:
p_injected: List[str] = p_raw.splitlines()
for (i, l) in reversed(list(enumerate(p_injected))):
    if l.startswith('import '):
        i += 1
        break
p_injected = '\n'.join(p_injected[:i]) + '\n\n' + '\n'.join('set_option ' + t.replace('=', ' ') for t in CORE_OPTIONS) + '\n\n' + '\n'.join(p_injected[i:])
print(p_injected)

import Mathlib

set_option maxHeartbeats 0
set_option maxRecDepth 100000
set_option tactic.hygienic false
set_option pp.fullNames true
set_option pp.funBinderTypes true
set_option pp.piBinderTypes true


open Nat

/--
There exists some positive integer `k` such that
`49 ∣ choose (2k, k)`.
We verify this by explicitly choosing `k = 25`.
-/
lemma exist_sol : ∃ k, 49 ∣ choose (2 * k) k := by
  use 25
  decide  -- `choose (50, 25) % 49 = 0` evaluates to true

/--
For all `k < 25`, `49` does **not** divide `choose (2k, k)`.
We express this using `k : Fin 25` to encode `k < 25` in the type.
We prove this by brute force using `decide`.
-/
lemma lowerbound : ∀ k : Fin 25, ¬ 49 ∣ choose (2 * k) k := by
  intro k
  fin_cases k <;> decide  -- checks all 0 ≤ k < 25

/--
Define the smallest `k` such that `49 ∣ choose (2k, k)`
using `Nat.find` on the existential witness from `exist_sol`.
-/
noncomputable def smallestKDivChoose : ℕ :=
  Nat.find (by
    -- Supply the witness from `exist_sol`
    exac

In [107]:
async with aiofiles.open(osp.join(
        '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Numina-Lean/lean',
        str(d['index'])+'.lean'
    ), 'r') as f:
    p_injected_orig = await f.read()

In [109]:
p_injected_orig == p_injected

True

In [110]:
d['parse_result']['units']

[{'i_begin': 16,
  'i_end': 43,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 43,
  'i_end': 73,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 73,
  'i_end': 106,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 106,
  'i_end': 135,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 135,
  'i_end': 169,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 169,
  'i_end': 204,
  'messages': [],
  'invocations': [],
  'goal_state': None,
  'goal_src_boundaries': None,
  'new_constants': None},
 {'i_begin': 204,
  'i_end': 214,
  'messages': [],
  'invocations': [],
  'goal_state': None

In [111]:
# units = await server.tactic_invocations_async(osp.join(parsing_dir, str(idx)+'.lean'))
# assert len(units) >= 1 and 'error' not in str([x.messages for x in units])

In [112]:
# units = [
#     u for u in units if len(u.invocations or []) > 0
# ]
# # assert len(units) == 1

In [113]:
# units[0]

In [114]:
parsed_units = [u for u in d['parse_result']['units'] if len(u['invocations'] or []) > 0]
print(len(parsed_units))

4


In [135]:
for u in parsed_units:
    break

In [136]:
invocations = u['invocations']

In [137]:
assert len(invocations[0]['before']) == 1

context_str, target = invocations[0]['before'][0].split('⊢ ')

lines = context_str.splitlines()
context = []
for l in lines:
    if not l.startswith(' '):   # Multi-line variable produced by pretty-printer
        context.append(l)
    else:
        context[-1] = context[-1] + '\n' + l

In [138]:
intros = []
for c in context:
    var_names = c.split(' : ')[0].split(' ')
    intros.extend(var_names)

In [139]:
tactic_header

'open Nat in\n'

In [140]:
hypotheses = ('∀' + '\n'.join('(' + c + ')' for c in context) + '\n, ') if len(context) > 0 else ''
formal_statement = hypotheses + target
assert '⊢' not in formal_statement
print(formal_statement)

init_state = await server.load_statement_async(formal_statement, intros=intros, header=load_header)
assert all(match_wo_mvar(g_parsed, str(g_now)) for g_parsed, g_now in zip(invocations[0]['before'], init_state.goals))

∃ (k : ℕ), 49 ∣ (2 * k).choose k


In [141]:
def is_deductive(ivc: dict) -> bool:
    return len(ivc['before']) == 1 and len(ivc['after']) == 1 and ivc['before'][0].split('⊢ ')[-1] == ivc['after'][0].split('⊢ ')[-1]

In [142]:
states = []
steps = []
cur_state = init_state

if is_deductive(invocations[0]):
    deductive_unit_indices = [0]
    idx = 0

    for i, ivc in enumerate(invocations[1:], 1):
        # if not is_deductive(ivc):
        #     print(f'Not deductive {i}')
        if is_deductive(ivc) and match_wo_mvar(ivc['before'][0], invocations[deductive_unit_indices[-1]]['after'][0]):
            # print(ivc['before'][0], '\n')
            deductive_unit_indices.append(i)
            if len(ivc['after'][0]) == 0:
                break

    for idx in deductive_unit_indices:
        ivc = invocations[idx]
        states.append(cur_state.goals[:])
        try:
            cur_state = await server.goal_tactic_async(cur_state, 0, ivc['tactic'])
            assert all(match_wo_mvar(g_parsed, str(g_now)) for g_parsed, g_now in zip(ivc['after'], cur_state.goals))
            steps.append(('', ivc['tactic']))
        except:
            cur_state = await server.goal_tactic_async(cur_state, 0, tactic_header + ivc['tactic'])
            assert all(match_wo_mvar(g_parsed, str(g_now)) for g_parsed, g_now in zip(ivc['after'], cur_state.goals))
            ivc
            steps.append((tactic_header, ivc['tactic']))
else:
    deductive_unit_indices = []


In [143]:
deductive_code_wo_space = remove_spaces(''.join(s[1] for s in steps))
code_segment = remove_comments(p_injected.encode()[u['i_begin']:u['i_end']].decode())

start_pos = None
for start_pos in re.finditer(r':=\s*by', code_segment):
    break
assert start_pos is not None
proof_code = code_segment[start_pos.span(0)[1]:]

assert remove_spaces(proof_code).startswith(deductive_code_wo_space), 'TODO: Severe error!'

ptr_deductive_code = 0
ptr_proof_line = None
proof_lines = proof_code.splitlines()
for (ptr_proof_line, line) in enumerate(proof_lines):
    line_wo_space = remove_spaces(line)
    if deductive_code_wo_space[ptr_deductive_code:].startswith(line_wo_space):
        ptr_deductive_code += len(line_wo_space)
    else:
        break

assert ptr_deductive_code == len(deductive_code_wo_space)
print('\n'.join(proof_lines[ptr_proof_line:]))


  use 25
  decide  



In [144]:
proof_state = cur_state

have_step = f'have h_submission: {cur_state.goals[0].target} := by {{\n' + '\n'.join(proof_lines[ptr_proof_line:]) + '\n}'
states.append(proof_state.goals[:])
try:
    proof_state = await server.goal_tactic_async(proof_state, 0, have_step)
    assert (len(proof_state.goals) == 1 and proof_state.goals[0].target == cur_state.goals[0].target)
    steps.append(('', have_step))
except:
    proof_state = await server.goal_tactic_async(proof_state, 0, tactic_header + have_step)
    assert (len(proof_state.goals) == 1 and proof_state.goals[0].target == cur_state.goals[0].target)
    steps.append((tactic_header, have_step))

states.append(proof_state.goals[:])
submit_step = 'exact h_submission'
try:
    proof_state = await server.goal_tactic_async(proof_state, 0, submit_step)
    assert proof_state.is_solved
    steps.append(('', submit_step))
except:
    proof_state = await server.goal_tactic_async(proof_state, 0, tactic_header + submit_step)
    assert proof_state.is_solved
    steps.append((tactic_header, submit_step))

In [ ]:
# while idx < len(invocations):
#     is_success = False
#     for idx_new in range(idx+1, len(invocations)):
#         if match_wo_mvar(invocations[idx_new]['before'][0], deductive_units[-1]['after'][0]):
#             deductive_units.append(invocations[idx_new])
#             idx = idx_new
#             break
#     if len(deductive_units[-1]['after']) == 0

In [ ]:
for d in data_success:
    units = [u for u in d['parse_result']['units'] if len(u['invocations'] or []) > 0]
    for u in units:
        for ivc in u['invocations']:
            modified_goals_before = [g for g in ivc['before'] if g not in ivc['after']]
            if len(modified_goals_before) > 1:  # before - after?
                print(ivc['tactic'] + '\n---\n')

In [ ]:
print('\n'.join(ivc['before']))
print()
print(ivc['tactic'])
print()
print('\n'.join(ivc['after']))

In [ ]:
invocations[0].before[0]

In [ ]:
print(init_state.goals[0])

In [ ]:
for ivc in invocations:
    print(ivc.tactic)
    print()

In [ ]:
def zip_strict(*args):
    assert len(args) > 1 and all(len(args[0]) == len(a) for a in args[1:])
    return zip(*args)

In [ ]:
statement = 'example\n' + '\n'.join()

In [ ]:
print(units[-1].invocations[0].before[0])

In [ ]:
# for k, v in units[-1].invocations[0].asdict().items():
ivc = units[-1].invocations[0]
print('# Before:\n', ivc.before)
print('# Tactic:\n', ivc.tactic)
print('# After:\n', ivc.after)

In [ ]:
parse_idents(ivc.tactic)

In [ ]:
for c in ivc.used_constants:
    if any(c.startswith(t+'.') for t in I.chain(open_list, open_scoped_list)):
        print(c)

In [ ]:
for c in ivc.used_constants:
    print(c)

In [ ]:
def parse_nested_invocations_inplace(invocations: List[TacticInvocation]) -> None:
    if len(invocations) == 1:
        return

    while len(invocations) > 1:
        length_before = len(invocations)
        for end_pos, ivc in enumerate(invocations):
            if ivc.after == []: # Detect goal end
                if not set(invocations[end_pos].before).issubset(set(invocations[end_pos-1].after)):    # The match fails at the beginning
                    continue
                for delta in range(2, end_pos+2):
                    if not set(invocations[end_pos-delta+1].before).issubset(set(invocations[end_pos-delta].after)): # current delta is not matched anymore
                        break
                start_pos = end_pos - delta + 1 # [start_pos, end_pos]
                
                # print(start_pos, end_pos)
                if start_pos == 0: # The whole sequence is matched and no whole block (no nested)
                    return
                else:
                    if len(invocations[start_pos-1].sub_invocations) != 0:
                        # Replacing old records with new records
                        pass

                    if all(re.sub(r'\s*', '', i.tactic) in re.sub(r'\s*', '', invocations[start_pos-1].tactic) for i in invocations[start_pos:end_pos+1]):  # Nested whole block
                        # All are sub-invocations
                        invocations[start_pos-1].sub_invocations.extend(invocations[start_pos:end_pos+1])
                        invocations = invocations[:start_pos] + invocations[end_pos+1:]
                    break
        if len(invocations) == length_before:   # No change: exit
            return

In [ ]:
parse_nested_invocations_inplace(invocations)

In [ ]:
invocations

In [ ]:
units[0]